In [41]:
import numpy as np
def MATRIX_FINDER(n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates):

    # N = n**2
    N = n + 1
    ones_vec = np.ones(N - 1)

    Kp = np.diag(np.append(k_positive_rates, 0))
    Km = np.append(np.diag(k_negative_rates), np.zeros((1, len(k_negative_rates))), axis=0)
    # print(Kp)
    # print(Km)
    
    Pp = np.diag(np.insert(p_positive_rates, 0, 0))
    Pm = np.vstack([np.zeros((1, len(p_negative_rates))), np.diag(p_negative_rates)])    # print("a", a)

    adjusted_alpha_mat = np.delete(alpha_matrix, -1, axis = 0)
    adjusted_beta_mat = np.delete(beta_matrix, 0, axis = 0)

    Da = np.diag(alpha_matrix[:-1, 1:] @ ones_vec)
    Db = np.diag(beta_matrix[1:, :-1] @ ones_vec)

    U = np.diag(k_negative_rates)
    I = np.diag(p_negative_rates)
    Q = Kp[:-1, :]
    D = np.delete(Pp, 0, axis=0)

    M_mat = U + Da
    N_mat = I + Db

    G = Km + adjusted_alpha_mat.T
    H = Pm + adjusted_beta_mat.T

    M_inv = np.linalg.inv(M_mat); N_inv = np.linalg.inv(N_mat)

    L1 = G @ M_inv @ Q - Kp; L2 = H @ N_inv @ D - Pp
    W1 = M_inv @ Q; W2 = N_inv @ D

    return Kp, Pp, G, H, Q, M_mat, D, N_mat, L1, L2, W1, W2

def MATTIA_FULL(t, state_array, n, Kp, Pp, G, H, Q, M_mat, D, N_mat):

    N = n + 1
    # N = n**2
    # ones_vec = np.ones(N - 1)
    ones_vec = np.ones((1, N-1), dtype = float) # shape (1, N-1)

    # assert len(state_array) == 3*N
    # print(state_array)
    a = state_array[0: N]
    b = state_array[N: 2*N - 1]
    c = state_array[2*N - 1: 3*N - 2]
    x = float(state_array[-2])
    y = float(state_array[-1])

    a_dot = (G @ b) + (H @ c) - x * (Kp @ a) - y * (Pp @ a)  
    b_dot = x * (Q @ a) - (M_mat @ b)
    c_dot = y * (D @ a) - (N_mat @ c)

    x_dot = -1*ones_vec.T @ b_dot
    y_dot = -1*ones_vec.T @ c_dot

    return np.concatenate((a_dot, b_dot, c_dot, np.array([x_dot, y_dot])))

def MATTIA_REDUCED(t, state_array, n, x_tot, y_tot, L1, L2, W1, W2):

    N = n + 1
    # N = n**2
    # ones_vec = np.ones(N - 1)
    ones_vec = np.ones((1, N-1), dtype = float) # shape (1, N-1)

    # assert len(state_array) == N

    a = state_array[0: N].astype(float)

    a_dot = ((x_tot * L1 @ a) / (1 + ones_vec @ W1 @ a)) + (y_tot * (L2 @ a) / (1 + ones_vec @ W2 @ a)) 

    return a_dot

def MATTIA_REDUCED_JACOBIAN(t, state_array, n, x_tot, y_tot, L1, L2, W1, W2):

    N = n + 1
    # N = n**2
    state_array = np.asarray(state_array, dtype=float)
    state_array = state_array.reshape((N, 1))  # shape (N, 1)

    ones_vec_j = np.ones((1, N-1), dtype = float) # shape (1, N-1)

    # Compute denominators
    denom1 = float(1 + np.dot(ones_vec_j, (W1 @ state_array)).item())
    denom2 = float(1 + np.dot(ones_vec_j, (W2 @ state_array)).item())

    term1 = (L1 / denom1) - (((L1 @ state_array) @ (ones_vec_j @ W1)) / (denom1**2))
    term2 = (L2 / denom2) - (((L2 @ state_array) @ (ones_vec_j @ W2)) / (denom2**2))

    J = (x_tot * term1) + (y_tot * term2)

    return J

class all_parameter_generation:
    """
    Generate state transitions and random parameters (a, b, c, enzyme) for an n-site phosphorylation model.

    Args:
        n: number of sites (int)
        distribution: distribution name ("gamma" supported)
        params: parameters for the distribution (for gamma: [shape, scale])
        verbose: if True, prints transitions and matrices
    """
    def __init__(self, n: int, reaction_types: str, distribution: str, distribution_paramaters: List[float], verbose: bool = False):
        self.n = n
        # self.num_states = n**2
        self.num_states = n + 1
        self.distribution = distribution
        self.params = distribution_paramaters
        self.reaction_types = reaction_types
        self.verbose = verbose
        self.rng = np.random.default_rng()
        
    @staticmethod
    def padded_binary(i: int, n: int) -> str:
        return bin(i)[2:].zfill(n)

    @staticmethod
    def binary_string_to_array(string: str) -> np.ndarray:
        return np.array([int(i) for i in string], dtype=int)

    def calculate_valid_transitions(self) -> Tuple[List[List[Any]], List[List[Any]]]:
        """
        Returns:
            valid_X_reactions: list of [state_i_str, state_j_str, i, j, "E"]
            valid_Y_reactions: list of [state_i_str, state_j_str, i, j, "F"]
        """
        all_states = [self.padded_binary(i, self.n) for i in range(self.num_states)]

        valid_difference_vectors: Set[Tuple[int, ...]] = set()
        valid_X_reactions: List[List[Any]] = []
        valid_Y_reactions: List[List[Any]] = []

        for i in range(self.num_states):
            arr_i = self.binary_string_to_array(all_states[i])
            for j in range(self.num_states):
                if i == j:
                    continue
                arr_j = self.binary_string_to_array(all_states[j])
                diff = arr_j - arr_i
                # if self.reaction_types == "distributive":
                    
                hamming_weight = np.sum(np.abs(diff))

                if hamming_weight == 1:
                    # +1 -> phosphorylation (E), -1 -> dephosphorylation (F)
                    element = "E" if np.any(diff == 1) else "F"
                    if element == "E":
                        if self.verbose:
                            print(f"{all_states[i]} --> {all_states[j]} (E), {i}, {j}")
                        valid_X_reactions.append([all_states[i], all_states[j], i, j, element])
                    else:
                        if self.verbose:
                            print(f"{all_states[i]} --> {all_states[j]} (F), {i}, {j}")
                        valid_Y_reactions.append([all_states[i], all_states[j], i, j, element])
                    valid_difference_vectors.add(tuple(diff))

        return valid_X_reactions, valid_Y_reactions
    
    def alpha_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray,
                                            Dict[int, List[int]], Dict[int, List[int]],
                                            Dict[int, List[int]], Dict[int, List[int]]]:
        
        # valid_X_reactions, valid_Y_reactions = self.calculate_valid_transitions()

        shape, scale = self.params

        # alpha_matrix = np.zeros((self.num_states, self.num_states))
        alpha_array = np.array([self.rng.gamma(shape, scale) for i in range(self.n)])
        alpha_matrix = np.diag(alpha_array, 1)
        
        # alpha_matrix = np.diag([self.rng.gamma(shape, scale)]*(min(self.num_states, self.num_states - 1)), 1)[:self.num_states, :self.num_states]
        # for _, _, i, j, _ in valid_X_reactions:

        #     alpha_matrix[i][j] = self.rng.gamma(shape, scale)

        return alpha_matrix


    def beta_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray,
                                            Dict[int, List[int]], Dict[int, List[int]],
                                            Dict[int, List[int]], Dict[int, List[int]]]:
        
        # valid_X_reactions, valid_Y_reactions = self.calculate_valid_transitions()

        shape, scale = self.params
        beta_array = np.array([self.rng.gamma(shape, scale) for i in range(self.n)])
        
        beta_matrix = np.diag(beta_array, -1)
        # beta_matrix = np.zeros((self.num_states, self.num_states))
        
        # for _, _, i, j, _ in valid_Y_reactions:

        #     beta_matrix[i][j] = self.rng.gamma(shape, scale)

        # beta_matrix = np.diag([self.rng.gamma(shape, scale)]*(min(self.num_states-1, self.num_states)), -1)[:self.num_states, :self.num_states]
        
        return beta_matrix
    
    def k_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray]:
        # if self.distribution != "gamma":
        #     raise NotImplementedError("Only 'gamma' distribution implemented for a_parameter_generation")
        shape, scale = self.params
        if self.distribution == "gamma":
            k_positive_rates = self.rng.gamma(shape, scale, self.num_states)
            k_negative_rates = self.rng.gamma(shape, scale, self.num_states)
        if self.distribution == "levy":
            k_positive_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
            k_negative_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)

        return k_positive_rates, k_negative_rates

    def p_parameter_generation(self) -> Tuple[np.ndarray, np.ndarray]:
        
        # if self.distribution != "gamma":
        #     raise NotImplementedError("Only 'gamma' distribution implemented for b_parameter_generation")
        shape, scale = self.params
        if self.distribution == "gamma":
            p_positive_rates = self.rng.gamma(shape, scale, self.num_states)
            p_negative_rates = self.rng.gamma(shape, scale, self.num_states)
        if self.distribution == "levy":
            p_positive_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)
            p_negative_rates = levy.rvs(loc=shape, scale=scale, size=self.num_states - 1, random_state=self.rng)

        return p_positive_rates, p_negative_rates
    
    def total_concentration_generation(self) -> Tuple[np.ndarray, np.ndarray]:
        
        # if self.distribution != "gamma":
        #     raise NotImplementedError("Only 'gamma' distribution implemented for b_parameter_generation")
        shape, scale = self.params
        if self.distribution == "gamma":
            x_tot_concentration = self.rng.gamma(shape, scale, 1)
            y_tot_concentration = self.rng.gamma(shape, scale, 1)

        return x_tot_concentration, y_tot_concentration
    
def ALL_GUESSES(N, guesses):

    rng = np.random.default_rng()

    # 1. Very sharp corners: one component dominates (alpha << 1)
    for _ in range(N):
        alpha_sharp = np.full(N, 0.1)  # Small alpha pushes to corners
        guess = rng.dirichlet(alpha_sharp)
        guesses.append(guess)  # Remove last component for reduced system
    
    # 2. Edges: two components share mass (set others to very small alpha)
    edge_samples_per_pair = 2
    for i in range(N):
        for j in range(i+1, N):
            for _ in range(edge_samples_per_pair):
                alpha_edge = np.full(N, 0.05)  # Very small for components not on edge
                alpha_edge[i] = 1.0  # Moderate for edge components
                alpha_edge[j] = 1.0
                guess = rng.dirichlet(alpha_edge)
                guesses.append(guess)
    
    # 3. Faces: k components share mass equally
    for n_active in range(2, N+1):
        for _ in range(2):  # 2 samples per face
            alpha_face = np.full(N, 0.05)
            alpha_face[:n_active] = 1.0
            guess = rng.dirichlet(alpha_face)
            guesses.append(guess)
    
    # 4. Uniform-ish distribution (alpha = 1 is uniform)
    for _ in range(3):
        guess = rng.dirichlet(np.ones(N))
        guesses.append(guess)
    
    # 5. Slightly perturbed corners (exact corners)
    for i in range(N):
        corner = np.zeros(N)
        corner[i] = 1.0
        guesses.append(corner)

    return guesses

def duplicate(candidate, collection, norm_tol):
    """Check if candidate is close to any vector in collection"""
    if len(collection) == 0:
        return False
    
    for v in collection:
        if (np.linalg.norm(np.array(v) - np.array(candidate)) < norm_tol) and np.allclose(v, candidate):
            return True
    return False

def main():
    n = 2
    gen_rates = all_parameter_generation(n, "distributive", "gamma", (0.123, 4.46e6), verbose = False)
    rate_min, rate_max = 1e-1, 1e7
    print(gen_rates.alpha_parameter_generation())
if __name__ == "__main__":
    main()

[[    0.          117.2019333     0.       ]
 [    0.            0.        64111.4071177]
 [    0.            0.            0.       ]]


In [ ]:
import numpy as np
import sympy as sp
from typing import List, Tuple, Dict, Any, Set
from scipy.stats import levy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from joblib import Parallel, delayed
import pickle
# from PHOS_FUNCTIONS import MATTIA_FULL, MATTIA_REDUCED, MATTIA_REDUCED_JACOBIAN, MATRIX_FINDER, ALL_GUESSES, duplicate, all_parameter_generation

def original_phosphorylation_reduced(t, state_array, n, x_tot, y_tot, L1, L2, W1, W2):

    N = n + 1
    # N = n**2
    # ones_vec = np.ones(N - 1)
    ones_vec = np.ones((1, N-1), dtype = float) # shape (1, N-1)

    # assert len(state_array) == N

    a = state_array[0: N].astype(float)

    a_dot = ((x_tot * L1 @ a) / (1 + ones_vec @ W1 @ a)) + (y_tot * (L2 @ a) / (1 + ones_vec @ W2 @ a)) 

    return a_dot

def original_phosphorylation_full(t, state_array, n, Kp, Km, Pp, Pm, alpha, beta, Da, Db):

    N = n + 1
    # ones_vec = np.ones(N - 1)
    ones_vec = np.ones((1, N-1), dtype = float) # shape (1, N-1)

    # assert len(state_array) == 3*N + 2
    # print(state_array)
    # a = state_array[0: N]
    # b = state_array[N: 2*N - 1]
    # c = state_array[2*N - 1: 3*N - 2]

    a = state_array[0: N]
    b = state_array[N: 2*N]; b[-1] = 0
    c = state_array[2*N: 3*N]; c[0] = 0

    x = float(state_array[-2])
    y = float(state_array[-1])

    a_dot = ((Km + alpha.T) @ b) + ((Pm + beta.T) @ c) - x * (Kp @ a) - y * (Pp @ a)  
    b_dot = x * (Kp @ a) - ((Km - Da) @ b); b_dot[-1] = 0
    c_dot = y * (Pp @ a) - ((Pm - Db) @ c); c_dot[0] = 0

    x_dot = -1*ones_vec.T @ b_dot
    y_dot = -1*ones_vec.T @ c_dot

    return np.concatenate((a_dot, b_dot, c_dot, np.array([x_dot, y_dot])))

def matrix_finder_original(n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates):

    # N = n**2
    N = n + 1
    ones_vec = np.ones(N)

    k_positive_rates[-1] = 0; k_negative_rates[-1] = 0
    p_positive_rates[0] = 0; p_negative_rates[0] = 0

    Kp = np.diag(k_positive_rates)
    Km = np.diag(k_negative_rates)
    assert Kp.shape == (N, N), f"Kp shape must be ({N}, {N})"
    assert Km.shape == (N, N), f"Km shape must be ({N}, {N})"
    Pp = np.diag(p_positive_rates)
    Pm = np.diag(p_negative_rates)  
    assert Pp.shape == (N, N), f"Pp shape must be ({N}, {N})"
    assert Pm.shape == (N, N), f"Pm shape must be ({N}, {N})"
    Da = np.diag(alpha_matrix @ ones_vec)
    Db = np.diag(beta_matrix @ ones_vec)
    assert Da.shape == (N, N), f"Da shape must be ({N}, {N})"
    assert Db.shape == (N, N), f"Db shape must be ({N}, {N})"
    return Kp, Km, Pp, Pm, alpha_matrix, beta_matrix, Da, Db

def stable_event(t, state_array, n, x_tot, y_tot, L1, L2, W1, W2):
    a_dot = MATTIA_REDUCED(t, state_array, n, x_tot, y_tot, L1, L2, W1, W2)
    # Avoid division by zero using a small epsilon
    denom = np.maximum(np.abs(state_array), 1e-12)
    rel_change = np.max(np.abs(a_dot / denom))
    return rel_change - 1e-4  # event when relative change < 1e-6

stable_event.terminal = True
stable_event.direction = 0

def phosphorylation_system_solver(parameters_tuple, initial_states_array, final_t, full_model, plot):

    n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates, a_tot, x_tot, y_tot = parameters_tuple
    N = n + 1
    # N = n**2

    # assert np.all(initial_states_array <= (a_tot + 1e-12))
    # assert np.all(initial_states_array >= 0)
    # assert np.all(initial_states_array <= a_tot)

    #### SCALING TIME
    initial_t = 0
    # final_t = 2000
    t_span = (initial_t, final_t)

    ###### OBTAINING ALL MATRICES
    Kp, Km, Pp, Pm, alpha_matrix, beta_matrix, Da, Db = matrix_finder_original(n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates)
    
    cmap = plt.get_cmap('tab10')
    def color_for_species(idx):
        return cmap(idx % cmap.N)
    
    # default
    abstol = 1e-10
    reltol = 1e-8

    # if full_model == False:

    #     assert len(initial_states_array) == N

    #     mattia_reduced_parameter_tuple = (n, x_tot, y_tot, L1, L2, W1, W2)

    #     # ‘DOP853’ is recommended for solving with high precision (low values of rtol and atol).
    #     # method = 'DOP853'
    #     # if stiff, you should use ‘Radau’ or ‘BDF’. ‘LSODA’ can also be a good universal choice, 
    #     # but it might be somewhat less convenient to work with as it wraps old Fortran code.
    #     # 'LSODA' does not accept array-like jacobian

    #     method = 'LSODA'
    #     sol = solve_ivp(MATTIA_REDUCED, t_span = t_span, y0=np.asarray(initial_states_array, dtype=float),
    #                 args = mattia_reduced_parameter_tuple, method = method, jac = MATTIA_REDUCED_JACOBIAN, 
    #                 atol = abstol, rtol = reltol)
    #     print(sol.message)
    #     # sol = solve_ivp(MATTIA_REDUCED, t_span = t_span, y0=np.asarray(initial_states_array, dtype=float),
    #     #             args = mattia_reduced_parameter_tuple, method = 'LSODA', jac = MATTIA_REDUCED_JACOBIAN, 
    #     #             events = stable_event, atol = abstol, rtol = reltol)
    #     a_solution_stack = np.stack([sol.y[i] for i in range(0, N)]) / a_tot

    #     if plot == True:
    #         plt.figure(figsize = (8, 6))
    #         plt.style.use('seaborn-v0_8-whitegrid')
    #         for i in range(a_solution_stack.shape[0]):
    #             color = color_for_species(i)
    #             plt.plot(sol.t, a_solution_stack[i], color=color, label = f"[$A_{i}]$", lw=4, alpha = 0.4)
    #             print(f"final A_{i} = {a_solution_stack[i][-1]}")
    #             plt.title(f"Plotting reduced phosphorylation dynamics for n = {n}")

    # assert len(initial_states_array) == 3*n + 2
    # # assert np.all(initial_states_array >= 1e-15)
    # assert np.all(initial_states_array >= 0)

    mattia_parameter_tuple = (n, Kp, Km, Pp, Pm, alpha_matrix, beta_matrix, Da, Db)
    # t_array = np.linspace(t_span[0], t_span[1], 500)
    sol = solve_ivp(original_phosphorylation_full, t_span = t_span, y0=np.asarray(initial_states_array, dtype=float), 
                    args = mattia_parameter_tuple, method = 'LSODA', atol = abstol, rtol = reltol)

    a_solution_stack = np.stack([sol.y[i] for i in range(0, N)]) / a_tot
    # b_solution_stack = np.stack([sol.y[i] for i in range(N, 2*N - 1)]) 
    # c_solution_stack = np.stack([sol.y[i] for i in range(2*N - 1, 3*N - 2)]) 
    # x_solution = sol.y[-2]
    # y_solution = sol.y[-1]
    # for i in range(b_solution_stack.shape[0]):
    #     color = color_for_species(i + a_solution_stack.shape[0])
    #     plt.plot(sol.t, b_solution_stack[i], color=color, label = f"$B_{i}$", lw=1.5, linestyle='-', alpha = 0.75)
    #     print(f"final B_{i} = {b_solution_stack[i][-1]}")

    # for i in range(c_solution_stack.shape[0]):
    #     color = color_for_species(i + a_solution_stack.shape[0] + b_solution_stack.shape[0] - 1)
    #     plt.plot(sol.t, c_solution_stack[i], color=color, label=f"$C_{i+1}$", lw=1, linestyle='--', alpha = 1)
    #     print(f"final C_{i+1} = {c_solution_stack[i][-1]}")

    # print(f"final X = {x_solution[-1]}")
    # print(f"final Y = {y_solution[-1]}")
    # print(a_solution_stack)

    for i in range(a_solution_stack.shape[0]):
        color = color_for_species(i)
        plt.plot(sol.t, a_solution_stack[i], color=color, label = f"$[A_{i}]$", lw=4, alpha = 0.4)
        print(f"final numerical fp of A_{i}: {a_solution_stack[i][-1]}")
        plt.title(f"full system dynamics for n = {n}")
    # plt.plot(sol.t, x_solution, color='black', label="$X$", lw=1.75, alpha = 0.75)
    # plt.plot(sol.t, y_solution, color='gray', label="$Y$", lw=1, alpha = 0.75)

    
    plt.ylabel("concentration")
    plt.xlabel("time")
    plt.minorticks_on()
    plt.tight_layout()
    plt.xlim(t_span[0] - 0.1, t_span[1] + 0.1)
    plt.ylim(-0.05, 1.1)
    plt.legend(frameon=False)
    plt.show()
    
    return sol.y.T[-1]

def fp_finder(sites_n, a_tot_value, x_tot_value, y_tot_value,
                      alpha_matrix, beta_matrix,
                      k_positive_rates, k_negative_rates,
                      p_positive_rates, p_negative_rates):
    N = sites_n + 1
    # N = sites_n**2
    a_tot_value = 1

    # Kp, Pp, G, H, Q, M_mat, D, N_mat, L1, L2, W1, W2 = MATRIX_FINDER(sites_n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates)
    Kp, Km, Pp, Pm, alpha, beta, Da, Db = matrix_finder_original(sites_n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates)
    norm_tol = 1e-8
    final_t = 100000
    grid_num = 8
    stable_points = []
    plot = True
    full_system = True
    guesses = []

    attempt_total_num = 10
    for i in range(attempt_total_num):
        guess = np.random.rand(N)
        guesses.append(guess / np.sum(guess))
    
    # guesses = ALL_GUESSES(N, guesses)

    for guess in guesses:
        initial_states_array = guess
        parameters_tuple = (sites_n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates, a_tot_value, x_tot_value, y_tot_value)            
        full_sol = phosphorylation_system_solver(parameters_tuple, initial_states_array, final_t, full_system, plot)

        residuals = original_phosphorylation_full(1, full_sol, sites_n, Kp, Km, Pp, Pm, alpha, beta, Da, Db)
        if np.max(np.abs(residuals)) > 1e-8:
            print("Point was not found to be a valid zero.")
            continue

        if duplicate(full_sol, stable_points, norm_tol) == True:
            print("Duplicate found, skipping")
            continue

        stable_points.append(full_sol)

    if len(stable_points) == 0:
        return np.array([])
        
    print(f"# of stable points found is {len(stable_points)}")

    return np.array(stable_points)

def process_sample_script(i,
                   sites_n,
                   a_tot_value,
                   x_tot_value_parameter_array,
                   y_tot_value_parameter_array,
                   alpha_matrix_parameter_array,
                   beta_matrix_parameter_array,
                   k_positive_parameter_array,
                   k_negative_parameter_array,
                   p_positive_parameter_array,
                   p_negative_parameter_array):
    
    k_positive_rates = k_positive_parameter_array[i]
    k_negative_rates = k_negative_parameter_array[i]
    p_positive_rates = p_positive_parameter_array[i]
    p_negative_rates = p_negative_parameter_array[i]
    x_tot_value = x_tot_value_parameter_array[i][0]
    y_tot_value = y_tot_value_parameter_array[i][0]

    alpha_matrix = alpha_matrix_parameter_array[i]
    beta_matrix = beta_matrix_parameter_array[i]
    # k_negative_rates = k_positive_rates
    # p_negative_rates = p_positive_rates
    # NORMALIZING
    # alpha_matrix = alpha_matrix_parameter_array[i] / np.mean(alpha_matrix_parameter_array[i])
    # beta_matrix = beta_matrix_parameter_array[i] / np.mean(beta_matrix_parameter_array[i])

    # rate_min, rate_max = 1e-1, 1e7

    # def matrix_clip(matrix):
    #     clipped = matrix.copy()
    #     mask = clipped != 0
    #     clipped[mask] = np.clip(clipped[mask], rate_min, rate_max)
    #     return clipped
    
    # # CLIPPING
    # alpha_matrix = matrix_clip(alpha_matrix); beta_matrix = matrix_clip(beta_matrix)

    multistable_results = None
    
    unique_stable_fp_array = fp_finder(sites_n, a_tot_value, x_tot_value, y_tot_value,
                                                     alpha_matrix, beta_matrix, k_positive_rates,
                                                     k_negative_rates, p_positive_rates, p_negative_rates)
    print(unique_stable_fp_array, len(unique_stable_fp_array))
    possible_steady_states = np.floor((sites_n + 2) / 2).astype(int)

    if len(unique_stable_fp_array) == 2:
        multistable_results = {
        "num_of_stable_states": len(unique_stable_fp_array),
        "stable_states": unique_stable_fp_array,
        "total_concentration_values": np.array([a_tot_value, x_tot_value, y_tot_value]),
        "alpha_matrix": alpha_matrix,
        "beta_matrix": beta_matrix,
        "k_positive": k_positive_rates,
        "k_negative": k_negative_rates,
        "p_positive": p_positive_rates,
        "p_negative": p_negative_rates
        }

    return multistable_results

def simulation_ode(sites_n, simulation_size):
    a_tot_value = 1
    N = sites_n + 1
    # N = sites_n**2
    gen_rates = all_parameter_generation(sites_n, "distributive", "gamma", (0.123, 4.46e6), verbose = False)
    rate_min, rate_max = 1e-1, 1e7
    alpha_matrix_parameter_array = np.array([gen_rates.alpha_parameter_generation() for _ in range(simulation_size)]) 
    beta_matrix_parameter_array = np.array([gen_rates.beta_parameter_generation() for _ in range(simulation_size)]) 
    # k_positive_parameter_array = np.array([np.ones(N - 1) for _ in range(simulation_size)])
    # k_negative_parameter_array = np.array([np.ones(N - 1) for _ in range(simulation_size)])
    # p_positive_parameter_array = np.array([np.ones(N - 1) for _ in range(simulation_size)])
    # p_negative_parameter_array = np.array([np.ones(N - 1) for _ in range(simulation_size)])
    k_positive_parameter_array = np.array([np.clip(gen_rates.k_parameter_generation()[0], rate_min, rate_max) for _ in range(simulation_size)])
    k_negative_parameter_array = np.array([np.clip(gen_rates.k_parameter_generation()[1], rate_min, rate_max) for _ in range(simulation_size)])
    p_positive_parameter_array = np.array([np.clip(gen_rates.p_parameter_generation()[0], rate_min, rate_max) for _ in range(simulation_size)])
    p_negative_parameter_array = np.array([np.clip(gen_rates.p_parameter_generation()[1], rate_min, rate_max) for _ in range(simulation_size)])
    gen_concentrations = all_parameter_generation(sites_n, "distributive", "gamma", (0.40637, 0.035587), verbose = False)
    concentration_min, concentration_max = 1e-4, 1e-1
    x_tot_value_parameter_array = np.array([np.clip(gen_concentrations.total_concentration_generation()[0], concentration_min, concentration_max) for _ in range(simulation_size)])
    y_tot_value_parameter_array = np.array([np.clip(gen_concentrations.total_concentration_generation()[1], concentration_min, concentration_max) for _ in range(simulation_size)])

    print(f"n={sites_n}, N={N}")
    print(f"Expected matrix shape: ({N-1}, {N-1})")
    print(f"alpha_matrix shape: {alpha_matrix_parameter_array[0].shape}")
    print(f"beta_matrix shape: {beta_matrix_parameter_array[0].shape}")
    results = Parallel(n_jobs=-2, backend="loky")(
        delayed(process_sample_script)(
            i,
            sites_n,
            a_tot_value,
            x_tot_value_parameter_array,
            y_tot_value_parameter_array,
            alpha_matrix_parameter_array,
            beta_matrix_parameter_array,
            k_positive_parameter_array,
            k_negative_parameter_array,
            p_positive_parameter_array,
            p_negative_parameter_array
        ) for i in range(simulation_size)
    )
    
    multistable_results_list = []

    for multi_res in results:
        if multi_res is not None:
            multistable_results_list.append(multi_res)
    multifile = f"multistability_parameters_{simulation_size}_{sites_n}.pkl"

    with open(multifile, "wb") as f:
        pickle.dump(multistable_results_list, f, protocol=pickle.HIGHEST_PROTOCOL)

    return

def main():
    n = 2
    # phosphorylation_system_solver()
    simulation_ode(n, 1)
    
if __name__ == "__main__":
    main()

n=2, N=3
Expected matrix shape: (2, 2)
alpha_matrix shape: (3, 3)
beta_matrix shape: (3, 3)


capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


IndexError: index -1 is out of bounds for axis 0 with size 0

In [55]:
import numpy as np
import sympy as sp
from typing import List, Tuple, Dict, Any, Set
from scipy.stats import levy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from joblib import Parallel, delayed
import pickle
# from PHOS_FUNCTIONS import MATTIA_FULL, MATTIA_REDUCED, MATTIA_REDUCED_JACOBIAN, MATRIX_FINDER, ALL_GUESSES, duplicate, all_parameter_generation

def original_phosphorylation_reduced(t, state_array, n, x_tot, y_tot, L1, L2, W1, W2):

    N = n + 1
    # N = n**2
    # ones_vec = np.ones(N - 1)
    ones_vec = np.ones((1, N-1), dtype = float) # shape (1, N-1)

    # assert len(state_array) == N

    a = state_array[0: N].astype(float)

    a_dot = ((x_tot * L1 @ a) / (1 + ones_vec @ W1 @ a)) + (y_tot * (L2 @ a) / (1 + ones_vec @ W2 @ a)) 

    return a_dot
def original_phosphorylation_full(t, state_array, n, Kp, Km, Pp, Pm, alpha, beta, Da, Db):
    """
    Full model:
    state_array layout: [a (N), b (N), c (N), x, y] => total length 3*N + 2 expected
    Be defensive: if a shorter vector is passed we handle it gracefully (avoid indexing errors).
    """
    N = n + 1
    ones_vec = np.ones((1, N-1), dtype=float)  # you used (1, N-1) earlier; keep same shape for sums

    state = np.asarray(state_array, dtype=float).copy()
    # If state is shorter than expected, raise a clear error
    expected_len = 3 * N + 2
    if state.size < expected_len:
        raise ValueError(f"original_phosphorylation_full expected state length {expected_len}, got {state.size}")

    # extract slices (they should exist now)
    a = state[0: N].copy()
    b = state[N: 2*N].copy()
    c = state[2*N: 3*N].copy()

    # safety: only set boundary entries when array non-empty
    if b.size > 0:
        b[-1] = 0.0
    if c.size > 0:
        c[0] = 0.0

    x = float(state[-2])
    y = float(state[-1])

    # do computations (retain your algebra)
    a_dot = ((Km + alpha.T) @ b) + ((Pm + beta.T) @ c) - x * (Kp @ a) - y * (Pp @ a)
    b_dot = x * (Kp @ a) - ((Km - Da) @ b)
    c_dot = y * (Pp @ a) - ((Pm - Db) @ c)

    # set boundary entries of derivatives if arrays non-empty
    if b_dot.size > 0:
        b_dot = b_dot.copy()
        b_dot[-1] = 0.0
    if c_dot.size > 0:
        c_dot = c_dot.copy()
        c_dot[0] = 0.0

    # x_dot and y_dot: you previously used ones_vec.T @ b_dot, keep same semantics
    # make sure shapes align (reshape b_dot to (N-1,) ?). We'll sum with ones:
    # produce sums over first N-1 entries (since ones_vec is (1, N-1))
    x_dot = -1.0 * np.sum(b_dot[: (N - 1)])
    y_dot = -1.0 * np.sum(c_dot[1:])  # original code set c_dot[0]=0 so summing from 1 to N-1

    return np.concatenate((a_dot, b_dot, c_dot, np.array([x_dot, y_dot])), axis=0)


def matrix_finder_original(n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates):

    # N = n**2
    N = n + 1
    ones_vec = np.ones(N)

    k_positive_rates[-1] = 0; k_negative_rates[-1] = 0
    p_positive_rates[0] = 0; p_negative_rates[0] = 0

    Kp = np.diag(k_positive_rates)
    Km = np.diag(k_negative_rates)
    assert Kp.shape == (N, N), f"Kp shape must be ({N}, {N})"
    assert Km.shape == (N, N), f"Km shape must be ({N}, {N})"
    Pp = np.diag(p_positive_rates)
    Pm = np.diag(p_negative_rates)  
    assert Pp.shape == (N, N), f"Pp shape must be ({N}, {N})"
    assert Pm.shape == (N, N), f"Pm shape must be ({N}, {N})"
    Da = np.diag(alpha_matrix @ ones_vec)
    Db = np.diag(beta_matrix @ ones_vec)
    assert Da.shape == (N, N), f"Da shape must be ({N}, {N})"
    assert Db.shape == (N, N), f"Db shape must be ({N}, {N})"
    return Kp, Km, Pp, Pm, alpha_matrix, beta_matrix, Da, Db

def stable_event(t, state_array, n, x_tot, y_tot, L1, L2, W1, W2):
    a_dot = MATTIA_REDUCED(t, state_array, n, x_tot, y_tot, L1, L2, W1, W2)
    # Avoid division by zero using a small epsilon
    denom = np.maximum(np.abs(state_array), 1e-12)
    rel_change = np.max(np.abs(a_dot / denom))
    return rel_change - 1e-4  # event when relative change < 1e-6

stable_event.terminal = True
stable_event.direction = 0

def phosphorylation_system_solver(parameters_tuple, initial_states_array, final_t, full_model, plot):

    n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates, a_tot, x_tot, y_tot = parameters_tuple
    N = n + 1
    # N = n**2

    # assert np.all(initial_states_array <= (a_tot + 1e-12))
    # assert np.all(initial_states_array >= 0)
    # assert np.all(initial_states_array <= a_tot)

    #### SCALING TIME
    initial_t = 0
    # final_t = 2000
    t_span = (initial_t, final_t)

    ###### OBTAINING ALL MATRICES
    Kp, Km, Pp, Pm, alpha_matrix, beta_matrix, Da, Db = matrix_finder_original(n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates)
    
    cmap = plt.get_cmap('tab10')
    def color_for_species(idx):
        return cmap(idx % cmap.N)
    
    # default
    abstol = 1e-10
    reltol = 1e-8

    # if full_model == False:

    #     assert len(initial_states_array) == N

    #     mattia_reduced_parameter_tuple = (n, x_tot, y_tot, L1, L2, W1, W2)

    #     # ‘DOP853’ is recommended for solving with high precision (low values of rtol and atol).
    #     # method = 'DOP853'
    #     # if stiff, you should use ‘Radau’ or ‘BDF’. ‘LSODA’ can also be a good universal choice, 
    #     # but it might be somewhat less convenient to work with as it wraps old Fortran code.
    #     # 'LSODA' does not accept array-like jacobian

    #     method = 'LSODA'
    #     sol = solve_ivp(MATTIA_REDUCED, t_span = t_span, y0=np.asarray(initial_states_array, dtype=float),
    #                 args = mattia_reduced_parameter_tuple, method = method, jac = MATTIA_REDUCED_JACOBIAN, 
    #                 atol = abstol, rtol = reltol)
    #     print(sol.message)
    #     # sol = solve_ivp(MATTIA_REDUCED, t_span = t_span, y0=np.asarray(initial_states_array, dtype=float),
    #     #             args = mattia_reduced_parameter_tuple, method = 'LSODA', jac = MATTIA_REDUCED_JACOBIAN, 
    #     #             events = stable_event, atol = abstol, rtol = reltol)
    #     a_solution_stack = np.stack([sol.y[i] for i in range(0, N)]) / a_tot

    #     if plot == True:
    #         plt.figure(figsize = (8, 6))
    #         plt.style.use('seaborn-v0_8-whitegrid')
    #         for i in range(a_solution_stack.shape[0]):
    #             color = color_for_species(i)
    #             plt.plot(sol.t, a_solution_stack[i], color=color, label = f"[$A_{i}]$", lw=4, alpha = 0.4)
    #             print(f"final A_{i} = {a_solution_stack[i][-1]}")
    #             plt.title(f"Plotting reduced phosphorylation dynamics for n = {n}")

    # assert len(initial_states_array) == 3*n + 2
    # # assert np.all(initial_states_array >= 1e-15)
    # assert np.all(initial_states_array >= 0)

    mattia_parameter_tuple = (n, Kp, Km, Pp, Pm, alpha_matrix, beta_matrix, Da, Db)
    # t_array = np.linspace(t_span[0], t_span[1], 500)
    sol = solve_ivp(original_phosphorylation_full, t_span = t_span, y0=np.asarray(initial_states_array, dtype=float), 
                    args = mattia_parameter_tuple, method = 'LSODA', atol = abstol, rtol = reltol)

    a_solution_stack = np.stack([sol.y[i] for i in range(0, N)]) / a_tot
    # b_solution_stack = np.stack([sol.y[i] for i in range(N, 2*N - 1)]) 
    # c_solution_stack = np.stack([sol.y[i] for i in range(2*N - 1, 3*N - 2)]) 
    # x_solution = sol.y[-2]
    # y_solution = sol.y[-1]
    # for i in range(b_solution_stack.shape[0]):
    #     color = color_for_species(i + a_solution_stack.shape[0])
    #     plt.plot(sol.t, b_solution_stack[i], color=color, label = f"$B_{i}$", lw=1.5, linestyle='-', alpha = 0.75)
    #     print(f"final B_{i} = {b_solution_stack[i][-1]}")

    # for i in range(c_solution_stack.shape[0]):
    #     color = color_for_species(i + a_solution_stack.shape[0] + b_solution_stack.shape[0] - 1)
    #     plt.plot(sol.t, c_solution_stack[i], color=color, label=f"$C_{i+1}$", lw=1, linestyle='--', alpha = 1)
    #     print(f"final C_{i+1} = {c_solution_stack[i][-1]}")

    # print(f"final X = {x_solution[-1]}")
    # print(f"final Y = {y_solution[-1]}")
    # print(a_solution_stack)

    for i in range(a_solution_stack.shape[0]):
        color = color_for_species(i)
        plt.plot(sol.t, a_solution_stack[i], color=color, label = f"$[A_{i}]$", lw=4, alpha = 0.4)
        print(f"final numerical fp of A_{i}: {a_solution_stack[i][-1]}")
        plt.title(f"full system dynamics for n = {n}")
    # plt.plot(sol.t, x_solution, color='black', label="$X$", lw=1.75, alpha = 0.75)
    # plt.plot(sol.t, y_solution, color='gray', label="$Y$", lw=1, alpha = 0.75)

    
    plt.ylabel("concentration")
    plt.xlabel("time")
    plt.minorticks_on()
    plt.tight_layout()
    plt.xlim(t_span[0] - 0.1, t_span[1] + 0.1)
    plt.ylim(-0.05, 1.1)
    plt.legend(frameon=False)
    plt.show()
    
    return sol.y.T[-1]

def fp_finder(sites_n, a_tot_value, x_tot_value, y_tot_value,
              alpha_matrix, beta_matrix,
              k_positive_rates, k_negative_rates,
              p_positive_rates, p_negative_rates):
    """
    Generate guesses for A (length N), expand each to full-state vector of length 3*N+2,
    then call the full-system solver. This avoids empty-slice errors.
    """
    N = sites_n + 1
    a_tot_value = 1

    # Build matrices needed for the full model
    Kp, Km, Pp, Pm, alpha, beta, Da, Db = matrix_finder_original(
        sites_n, alpha_matrix, beta_matrix, k_positive_rates, k_negative_rates, p_positive_rates, p_negative_rates
    )

    norm_tol = 1e-8
    final_t = 100
    stable_points = []
    plot = True
    full_system = True   # <- run the full model (expects full-state initial condition)

    # create some random normalized guesses for A (length N)
    attempt_total_num = 10
    guesses = []
    for i in range(attempt_total_num):
        guess = np.random.rand(N)
        guesses.append(guess)

    for a_guess in guesses:
        # Expand the reduced guess (A only) into full initial state for the full model.
        # We'll place guess into A, set B and C to zero vectors, and set X,Y to the provided totals.
        b_init = np.zeros(N, dtype=float)
        c_init = np.zeros(N, dtype=float)
        x_init = float(x_tot_value)
        y_init = float(y_tot_value)

        initial_states_array = np.concatenate((a_guess, b_init, c_init, np.array([x_init, y_init])), axis=0)

        parameters_tuple = (
            sites_n,
            alpha_matrix,
            beta_matrix,
            k_positive_rates,
            k_negative_rates,
            p_positive_rates,
            p_negative_rates,
            a_tot_value,
            x_tot_value,
            y_tot_value,
        )

        # Call full-model solver
        full_sol = phosphorylation_system_solver(parameters_tuple, initial_states_array, final_t, full_system, plot)

        # If solver failed, skip
        if full_sol is None or full_sol.size == 0:
            print("Solver failed or returned empty solution; skipping this guess.")
            continue

        # Residual check (should be near zero vector)
        residuals = original_phosphorylation_full(1.0, full_sol, sites_n, Kp, Km, Pp, Pm, alpha, beta, Da, Db)
        if np.max(np.abs(residuals)) > 1e-8:
            print("Point was not found to be a valid zero (residual > 1e-8).")
            continue

        # duplicate detection: duplicate expects full-sol dimension as used here
        if duplicate(full_sol, stable_points, norm_tol):
            print("Duplicate found, skipping")
            continue

        stable_points.append(full_sol)

    if len(stable_points) == 0:
        return np.array([])

    print(f"# of stable points found is {len(stable_points)}")
    return np.array(stable_points)

def process_sample_script(i,
                   sites_n,
                   a_tot_value,
                   x_tot_value_parameter_array,
                   y_tot_value_parameter_array,
                   alpha_matrix_parameter_array,
                   beta_matrix_parameter_array,
                   k_positive_parameter_array,
                   k_negative_parameter_array,
                   p_positive_parameter_array,
                   p_negative_parameter_array):
    
    k_positive_rates = k_positive_parameter_array[i]
    k_negative_rates = k_negative_parameter_array[i]
    p_positive_rates = p_positive_parameter_array[i]
    p_negative_rates = p_negative_parameter_array[i]
    x_tot_value = x_tot_value_parameter_array[i][0]
    y_tot_value = y_tot_value_parameter_array[i][0]

    alpha_matrix = alpha_matrix_parameter_array[i]
    beta_matrix = beta_matrix_parameter_array[i]
    # k_negative_rates = k_positive_rates
    # p_negative_rates = p_positive_rates
    # NORMALIZING
    # alpha_matrix = alpha_matrix_parameter_array[i] / np.mean(alpha_matrix_parameter_array[i])
    # beta_matrix = beta_matrix_parameter_array[i] / np.mean(beta_matrix_parameter_array[i])

    # rate_min, rate_max = 1e-1, 1e7

    # def matrix_clip(matrix):
    #     clipped = matrix.copy()
    #     mask = clipped != 0
    #     clipped[mask] = np.clip(clipped[mask], rate_min, rate_max)
    #     return clipped
    
    # # CLIPPING
    # alpha_matrix = matrix_clip(alpha_matrix); beta_matrix = matrix_clip(beta_matrix)

    multistable_results = None
    
    unique_stable_fp_array = fp_finder(sites_n, a_tot_value, x_tot_value, y_tot_value,
                                                     alpha_matrix, beta_matrix, k_positive_rates,
                                                     k_negative_rates, p_positive_rates, p_negative_rates)
    print(unique_stable_fp_array, len(unique_stable_fp_array))
    possible_steady_states = np.floor((sites_n + 2) / 2).astype(int)

    if len(unique_stable_fp_array) == 2:
        multistable_results = {
        "num_of_stable_states": len(unique_stable_fp_array),
        "stable_states": unique_stable_fp_array,
        "total_concentration_values": np.array([a_tot_value, x_tot_value, y_tot_value]),
        "alpha_matrix": alpha_matrix,
        "beta_matrix": beta_matrix,
        "k_positive": k_positive_rates,
        "k_negative": k_negative_rates,
        "p_positive": p_positive_rates,
        "p_negative": p_negative_rates
        }

    return multistable_results

def simulation_ode(sites_n, simulation_size):
    a_tot_value = 1
    N = sites_n + 1
    # N = sites_n**2
    gen_rates = all_parameter_generation(sites_n, "distributive", "gamma", (0.123, 4.46e6), verbose = False)
    rate_min, rate_max = 1e-1, 1e7
    alpha_matrix_parameter_array = np.array([gen_rates.alpha_parameter_generation() for _ in range(simulation_size)]) 
    beta_matrix_parameter_array = np.array([gen_rates.beta_parameter_generation() for _ in range(simulation_size)]) 
    # k_positive_parameter_array = np.array([np.ones(N - 1) for _ in range(simulation_size)])
    # k_negative_parameter_array = np.array([np.ones(N - 1) for _ in range(simulation_size)])
    # p_positive_parameter_array = np.array([np.ones(N - 1) for _ in range(simulation_size)])
    # p_negative_parameter_array = np.array([np.ones(N - 1) for _ in range(simulation_size)])
    k_positive_parameter_array = np.array([np.clip(gen_rates.k_parameter_generation()[0], rate_min, rate_max) for _ in range(simulation_size)])
    k_negative_parameter_array = np.array([np.clip(gen_rates.k_parameter_generation()[1], rate_min, rate_max) for _ in range(simulation_size)])
    p_positive_parameter_array = np.array([np.clip(gen_rates.p_parameter_generation()[0], rate_min, rate_max) for _ in range(simulation_size)])
    p_negative_parameter_array = np.array([np.clip(gen_rates.p_parameter_generation()[1], rate_min, rate_max) for _ in range(simulation_size)])
    gen_concentrations = all_parameter_generation(sites_n, "distributive", "gamma", (0.40637, 0.035587), verbose = False)
    concentration_min, concentration_max = 1e-4, 1e-1
    x_tot_value_parameter_array = np.array([np.clip(gen_concentrations.total_concentration_generation()[0], concentration_min, concentration_max) for _ in range(simulation_size)])
    y_tot_value_parameter_array = np.array([np.clip(gen_concentrations.total_concentration_generation()[1], concentration_min, concentration_max) for _ in range(simulation_size)])

    print(f"n={sites_n}, N={N}")
    print(f"Expected matrix shape: ({N-1}, {N-1})")
    print(f"alpha_matrix shape: {alpha_matrix_parameter_array[0].shape}")
    print(f"beta_matrix shape: {beta_matrix_parameter_array[0].shape}")
    results = Parallel(n_jobs=-2, backend="loky")(
        delayed(process_sample_script)(
            i,
            sites_n,
            a_tot_value,
            x_tot_value_parameter_array,
            y_tot_value_parameter_array,
            alpha_matrix_parameter_array,
            beta_matrix_parameter_array,
            k_positive_parameter_array,
            k_negative_parameter_array,
            p_positive_parameter_array,
            p_negative_parameter_array
        ) for i in range(simulation_size)
    )
    
    multistable_results_list = []

    for multi_res in results:
        if multi_res is not None:
            multistable_results_list.append(multi_res)
    multifile = f"multistability_parameters_{simulation_size}_{sites_n}.pkl"

    with open(multifile, "wb") as f:
        pickle.dump(multistable_results_list, f, protocol=pickle.HIGHEST_PROTOCOL)

    return

def main():
    n = 2
    # phosphorylation_system_solver()
    simulation_ode(n, 1)
    
if __name__ == "__main__":
    main()

n=2, N=3
Expected matrix shape: (2, 2)
alpha_matrix shape: (3, 3)
beta_matrix shape: (3, 3)


/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:55: RuntimeWarning: overflow encountered in multiply
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:57: RuntimeWarning: overflow encountered in multiply
/Users/mattia.altomare/research/initial_code/.venv/lib/python3.13/site-packages/scipy/integrate/_ivp/lsoda.py:161: UserWarning: lsoda: Repeated convergence failures (perhaps bad Jacobian or tolerances).
  solver._y, solver.t = integrator.run(


final numerical fp of A_0: 7.226032619350235e+85
final numerical fp of A_1: 7.211934533405853e+85
final numerical fp of A_2: -1.5512928333576692e+82
Figure(640x480)
Point was not found to be a valid zero (residual > 1e-8).


/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:55: RuntimeWarning: invalid value encountered in matmul
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:56: RuntimeWarning: invalid value encountered in matmul
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:57: RuntimeWarning: invalid value encountered in matmul
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:55: RuntimeWarning: invalid value encountered in subtract
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:55: RuntimeWarning: overflow encountered in multiply
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:56: RuntimeWarning: overflow encountered in multiply
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:57: RuntimeWarning: overflow encountered in multiply
/Users/mattia.altomare/research/initial_code/.venv/lib/python3.13/sit

final numerical fp of A_0: nan
final numerical fp of A_1: nan
final numerical fp of A_2: nan
Figure(640x480)


/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:55: RuntimeWarning: overflow encountered in matmul
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:57: RuntimeWarning: overflow encountered in matmul
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:57: RuntimeWarning: invalid value encountered in subtract
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:55: RuntimeWarning: invalid value encountered in multiply
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:56: RuntimeWarning: invalid value encountered in multiply
/var/folders/2h/xz8d592j62n7f363_sbw7ly00000gn/T/ipykernel_48803/494929584.py:57: RuntimeWarning: invalid value encountered in multiply


final numerical fp of A_0: nan
final numerical fp of A_1: nan
final numerical fp of A_2: nan
Figure(640x480)


/Users/mattia.altomare/research/initial_code/.venv/lib/python3.13/site-packages/scipy/integrate/_ivp/lsoda.py:161: UserWarning: lsoda: Repeated convergence failures (perhaps bad Jacobian or tolerances).
  solver._y, solver.t = integrator.run(


final numerical fp of A_0: 2.455195646842606e+99
final numerical fp of A_1: 2.4504055274143096e+99
final numerical fp of A_2: -5.2708417081299585e+95
Figure(640x480)
Point was not found to be a valid zero (residual > 1e-8).
final numerical fp of A_0: 4.811118768053754e+85
final numerical fp of A_1: 4.8017322112177126e+85
final numerical fp of A_2: -1.032856403848545e+82
Figure(640x480)
Point was not found to be a valid zero (residual > 1e-8).
final numerical fp of A_0: nan
final numerical fp of A_1: nan
final numerical fp of A_2: nan
Figure(640x480)
final numerical fp of A_0: nan
final numerical fp of A_1: nan
final numerical fp of A_2: nan
Figure(640x480)
final numerical fp of A_0: 1.8473983743854607e+93
final numerical fp of A_1: 1.8437940755360687e+93
final numerical fp of A_2: -3.966015667942657e+89
Figure(640x480)
Point was not found to be a valid zero (residual > 1e-8).
final numerical fp of A_0: 5.88942809490354e+103
final numerical fp of A_1: 5.877937742199784e+103
final numeri